In [2]:
import torch
print("GPU is availble : -> " , torch.cuda.is_available())

GPU is availble : ->  True


In [3]:
from transformers import pipeline
import pandas as pd


In [4]:
# Pipeline ko correct spelling se recreate karo
# Load English to Urdu translator
translator = pipeline(
    "translation",
    model="Helsinki-NLP/opus-mt-en-de" , device = 0 # English → Urdu ONLY
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [5]:
# first data clean like remove symble which  not a text or also remove the space

def clean_data(text):
 try:
  if pd.isna(text) or text.strip() == '':
    return ''
  result = translator(str(text))
  return result
 except Exception as e:
  print(f"some thing erong {e}")
  return e

In [7]:
df = pd.read_csv("/content/en_test_with_sarcasm.csv")

In [8]:
df.columns
len(df)

315

In [9]:
file_name = "/content/en_test_with_sarcasm.csv"
text = "text"
dev_sarcasm_english_german = "test_sarcasm_english_german.csv"

In [10]:
translate_into_german = []
total_text = len(df)
batch_size = 50

for i in range(0, total_text, batch_size):
    batch_end = min(i + batch_size, total_text)
    batch_text = df["text"].iloc[i:batch_end].tolist()

    batch_result_german = translator(
        batch_text,
        src_lang="en_XX",    # English source
        tgt_lang="de_DE"     # ✅ Correct Urdu code
    )

    print(f"processing ......{i + 1} to .....{batch_end}")

    translated_texts = [result['translation_text'] for result in batch_result_german]
    translate_into_german.extend(translated_texts)

    progress = ((i + batch_size) / total_text) * 100
    print(f"✅ Progress: {min(progress, 100):.1f}% complete")

processing ......1 to .....50
✅ Progress: 15.9% complete
processing ......51 to .....100
✅ Progress: 31.7% complete
processing ......101 to .....150
✅ Progress: 47.6% complete
processing ......151 to .....200
✅ Progress: 63.5% complete
processing ......201 to .....250
✅ Progress: 79.4% complete
processing ......251 to .....300
✅ Progress: 95.2% complete
processing ......301 to .....315
✅ Progress: 100.0% complete


In [11]:
df["translate_german"] = translate_into_german
file_save = ["binary" , "multiclass" , "translate_german"]
print(f"column save{file_save}")

column save['binary', 'multiclass', 'translate_german']


In [12]:

final_csv_file = df[file_save].copy()  # Add ()
final_csv_file.to_csv(dev_sarcasm_english_german, index=False)  # Fix False

In [ ]:

print(f"📁 Output saved as: {dev_sarcasm_english_german}")